In [1]:
import re
import os
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Activation, Dense, Embedding, Flatten, Dropout, Conv1D, MaxPooling1D

from nltk.corpus import stopwords 

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import nltk ##불용어 처리를 위해
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    # TPU 출력
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 
    # TPU 분산 전략 출력. 없으면 CPU
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.64.148.122:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.64.148.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.64.148.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [5]:
#########텍스트 전처리################################

In [6]:
df = pd.read_csv('/content/gdrive/MyDrive/해커톤/소설작가/train.csv')
df.head(10)

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
5,5,"""It was well fought,"" he said, ""and, by my soo...",4
6,6,"Not to pay him was impossible, considering his...",3
7,7,"“A proper figure of a man at-arms,” said the l...",2
8,8,"'You were not here last Sunday night,' he said.",0
9,9,“You must not ask me that!” I cried. “Hell may...,4


In [7]:
df['text'] = df['text'].apply(lambda x:x.lower())#소문자로

In [8]:
df.head(10)

,index,text,author
0,0,"he was almost choking. there was so much, so m...",3
1,1,"“your sister asked for it, i suppose?”",2
2,2,"she was engaged one day as she walked, in per...",1
3,3,"the captain was in the porch, keeping himself ...",4
4,4,"“have mercy, gentlemen!” odin flung up his han...",3
5,5,"""it was well fought,"" he said, ""and, by my soo...",4
6,6,"not to pay him was impossible, considering his...",3
7,7,"“a proper figure of a man at-arms,” said the l...",2
8,8,"'you were not here last sunday night,' he said.",0
9,9,“you must not ask me that!” i cried. “hell may...,4


In [9]:
def clean_text(text):
    return re.sub('[^a-zA-Z]', ' ', text)#알파벳 제외 공백
def d_space(text):
    return re.sub(' +', ' ', text)#공백으로 된것을 다시 공백 한칸으로

In [10]:
df['text'] = df['text'].apply(clean_text)

In [11]:
df.head(10)

,index,text,author
0,0,he was almost choking there was so much so m...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in per...,1
3,3,the captain was in the porch keeping himself ...,4
4,4,have mercy gentlemen odin flung up his han...,3
5,5,it was well fought he said and by my soo...,4
6,6,not to pay him was impossible considering his...,3
7,7,a proper figure of a man at arms said the l...,2
8,8,you were not here last sunday night he said,0
9,9,you must not ask me that i cried hell may...,4


In [12]:
df['text'] = df['text'].apply(d_space)

In [13]:
df.head(10)

,index,text,author
0,0,he was almost choking there was so much so muc...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in peru...,1
3,3,the captain was in the porch keeping himself c...,4
4,4,have mercy gentlemen odin flung up his hands ...,3
5,5,it was well fought he said and by my sooth th...,4
6,6,not to pay him was impossible considering his ...,3
7,7,a proper figure of a man at arms said the lit...,2
8,8,you were not here last sunday night he said,0
9,9,you must not ask me that i cried hell may hav...,4


In [14]:
def stop_words(text):   # 불용어 처리 
    stop_words = list(set(stopwords.words('english')))
#     stop_words = stop_words + ['said','one','would','could','mr','man','know','like','see','must'
#                                 'come','us','say','may','thouugh','think','made','go','came','might',
#                                 'sir','mrs','miss','looked','make','went','take','look','tell','asked',
#                                 'saw','seemed','took','put','looking','give','get']
    lst = text.split(' ')
    r = [x for x in lst if x not in stop_words]
    return ' '.join(r)

In [15]:
df['text'] = df['text'].apply(stop_words)

In [16]:
df.head(10)

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing jane last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands write anyway...,3
5,5,well fought said sooth charge us twice,4
6,6,pay impossible considering character talk fell...,3
7,7,proper figure man arms said little knight man...,2
8,8,last sunday night said,0
9,9,must ask cried hell may noble flames known sc...,4


In [17]:
df['text'] = df['text'].apply(lambda x:x.strip())#문자열의 앞에 \n과 공백 제거

In [18]:
df.head(10)

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing jane last lett...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands write anyway ...,3
5,5,well fought said sooth charge us twice,4
6,6,pay impossible considering character talk fell...,3
7,7,proper figure man arms said little knight man ...,2
8,8,last sunday night said,0
9,9,must ask cried hell may noble flames known sco...,4


In [19]:
#####모델 입력 전 토큰화 및 시퀀스화##########

In [20]:
X_train= df[['text']]
y_train= df['author']

In [21]:
X_train=X_train['text']

In [22]:
classes = df['author'].nunique()

In [23]:
classes#클래스 갯수

5

In [24]:
max_len = 256
embedding_dim = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [25]:
word_index = tokenizer.word_index#X_train의 나온 단어들에 대해서 토큰화가 되고 그 토큰들에 대해서 인덱스가 매겨짐을 확인
print(word_index)

{'odin': 1, 'said': 2, 'one': 3, 'would': 4, 'mr': 5, 'could': 6, 'man': 7, 'upon': 8, 'know': 9, 'well': 10, 'time': 11, 'little': 12, 'see': 13, 'come': 14, 'like': 15, 'must': 16, 'good': 17, 'say': 18, 'much': 19, 'think': 20, 'us': 21, 'never': 22, 'cried': 23, 'may': 24, 'sir': 25, 'two': 26, 'go': 27, 'though': 28, 'nothing': 29, 'made': 30, 'old': 31, 'came': 32, 'great': 33, 'last': 34, 'way': 35, 'thought': 36, 'day': 37, 'might': 38, 'first': 39, 'hand': 40, 'shall': 41, 'back': 42, 'even': 43, 'long': 44, 'face': 45, 'away': 46, 'mrs': 47, 'room': 48, 'house': 49, 'without': 50, 'every': 51, 'miss': 52, 'still': 53, 'eyes': 54, 'looked': 55, 'asked': 56, 'tell': 57, 'went': 58, 'make': 59, 'young': 60, 'take': 61, 'quite': 62, 'head': 63, 'something': 64, 'look': 65, 'yes': 66, 'dear': 67, 'night': 68, 'yet': 69, 'let': 70, 'door': 71, 'another': 72, 'oh': 73, 'ever': 74, 'life': 75, 'mind': 76, 'moment': 77, 'saw': 78, 'prince': 79, 'heard': 80, 'always': 81, 'father': 82,

In [26]:
sequences = tokenizer.texts_to_sequences(X_train)#위에서 토큰화가 된 문장들을 시퀀스 형태로 변환
X_train = sequence.pad_sequences(sequences, maxlen = max_len)

In [27]:
X_train#확인

array([[   0,    0,    0, ...,    1, 1168, 2819],
       [   0,    0,    0, ...,  180,   56,  214],
       [   0,    0,    0, ..., 5386,  281,    2],
       ...,
       [   0,    0,    0, ...,  180, 1421,    1],
       [   0,    0,    0, ...,  309, 3562,  153],
       [   0,    0,    0, ...,    2,   66,   15]], dtype=int32)

In [29]:
label_encoder = preprocessing.LabelEncoder()#라벨 원핫인코딩,라벨이 문자일때 사용 이미 author이 숫자로 나뉘어져있어 필요없어보임
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train)

In [40]:
y_train

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [30]:
num_words = len(word_index) + 1

In [ ]:
####모델링######

In [31]:
model = Sequential()

In [32]:
model.add(Embedding(num_words,
                    embedding_dim,
                    input_length = max_len))#텍스트를 넣기 위한 층

In [33]:
model.add(Conv1D(filters = 50, 
                 kernel_size = 5, 
                 strides = 1, 
                 padding = 'valid'))#1D 합성곱층

In [34]:
model.add(MaxPooling1D(1, padding = 'valid'))#풀링

In [35]:
model.add(Flatten()) #완전 연결 계층
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(classes, activation = 'softmax'))

In [36]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'Adam',
              metrics = ['acc'])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 256, 300)          10051800  
_________________________________________________________________
conv1d (Conv1D)              (None, 252, 50)           75050     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 252, 50)           0         
_________________________________________________________________
flatten (Flatten)            (None, 12600)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               6451712   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2

In [38]:
history = model.fit(x = X_train, 
                    y = y_train, 
                    batch_size = 128, 
                    epochs = 5, 
                    verbose = 1, 
                    validation_split = 0.1)

Epoch 1/5
386/386 [==============================] - 26s 65ms/step - loss: 1.0370 - acc: 0.5829 - val_loss: 0.7366 - val_acc: 0.7221
Epoch 2/5
386/386 [==============================] - 24s 63ms/step - loss: 0.5127 - acc: 0.8122 - val_loss: 0.7559 - val_acc: 0.7218
Epoch 3/5
386/386 [==============================] - 24s 63ms/step - loss: 0.3013 - acc: 0.8912 - val_loss: 0.8890 - val_acc: 0.7137
Epoch 4/5
386/386 [==============================] - 24s 63ms/step - loss: 0.1947 - acc: 0.9303 - val_loss: 1.0970 - val_acc: 0.7059
Epoch 5/5
386/386 [==============================] - 24s 63ms/step - loss: 0.1459 - acc: 0.9469 - val_loss: 1.2731 - val_acc: 0.6928


In [ ]:
######모델링2########

In [89]:
model2 = Sequential()

In [90]:
model2.add(Embedding(num_words,
                    embedding_dim,
                    input_length = max_len))#텍스트를 넣기 위한 층

In [91]:
model2.add(Conv1D(filters = 128, 
                 kernel_size = 9, 
                 strides = 1, 
                 padding = 'same'
                ))#1D 합성곱층

In [92]:
model2.add(Activation(activation='relu'))

In [93]:
model2.add(Dropout(0.3))

In [94]:
model2.add(Conv1D(filters = 256, 
                 kernel_size = 6, 
                 strides = 1, 
                 padding = 'same'
                ))#1D 합성곱층

In [95]:
model2.add(Activation(activation='relu'))

In [96]:
model2.add(Dropout(0.4))

In [97]:
model2.add(Conv1D(filters = 128, 
                 kernel_size = 3, 
                 strides = 1, 
                 padding = 'same'
                ))#1D 합성곱층

In [98]:
model2.add(Activation(activation='relu'))

In [99]:
model2.add(Dropout(0.5))

In [100]:
model2.add(keras.layers.GlobalAveragePooling1D())

In [101]:
model2.add(Flatten()) #완전 연결 계층
model2.add(Dense(512, activation = 'relu'))
model2.add(Dropout(0.1))
model2.add(Dense(classes, activation = 'softmax'))

In [102]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 256, 300)          10051800  
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 256, 128)          345728    
_________________________________________________________________
activation_9 (Activation)    (None, 256, 128)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 256, 128)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 256, 256)          196864    
_________________________________________________________________
activation_10 (Activation)   (None, 256, 256)          0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 256, 256)         

In [103]:
model2.compile(loss = 'categorical_crossentropy', 
              optimizer = 'Adam',
              metrics = ['acc'])

In [104]:
history = model2.fit(x = X_train, 
                    y = y_train, 
                    batch_size = 128, 
                    epochs = 8, 
                    verbose = 1, 
                    validation_split = 0.1)

Epoch 1/8
386/386 [==============================] - 58s 147ms/step - loss: 1.3275 - acc: 0.4138 - val_loss: 1.0316 - val_acc: 0.5876
Epoch 2/8
386/386 [==============================] - 56s 145ms/step - loss: 0.7905 - acc: 0.6991 - val_loss: 0.7327 - val_acc: 0.7269
Epoch 3/8
386/386 [==============================] - 56s 146ms/step - loss: 0.5079 - acc: 0.8163 - val_loss: 0.7548 - val_acc: 0.7316
Epoch 4/8
386/386 [==============================] - 56s 146ms/step - loss: 0.3796 - acc: 0.8644 - val_loss: 0.8714 - val_acc: 0.7256
Epoch 5/8
386/386 [==============================] - 56s 146ms/step - loss: 0.3036 - acc: 0.8917 - val_loss: 1.0086 - val_acc: 0.7165
Epoch 6/8
386/386 [==============================] - 56s 146ms/step - loss: 0.2490 - acc: 0.9132 - val_loss: 1.1318 - val_acc: 0.7187
Epoch 7/8
386/386 [==============================] - 56s 146ms/step - loss: 0.2169 - acc: 0.9233 - val_loss: 1.1870 - val_acc: 0.7050
Epoch 8/8
386/386 [==============================] - 56s 145ms

In [ ]:
#####모델링3###########

In [180]:
model3 = Sequential()

In [181]:
model3.add(Embedding(num_words,
                    embedding_dim,
                    input_length = max_len))#텍스트를 넣기 위한 층

In [182]:
model3.add(Conv1D(filters = 64, 
                 kernel_size = 9, 
                 strides = 1, 
                 padding = 'valid'
                ))#1D 합성곱층

In [183]:
model3.add(Activation(activation='relu'))

In [184]:
model3.add(Dropout(0.3))

In [185]:
model3.add(Conv1D(filters = 128, 
                 kernel_size = 6, 
                 strides = 1, 
                 padding = 'valid'
                ))#1D 합성곱층

In [186]:
model3.add(Activation(activation='relu'))

In [187]:
model3.add(Dropout(0.4))

In [188]:
model3.add(Conv1D(filters = 64, 
                 kernel_size = 3, 
                 strides = 1, 
                 padding = 'valid'
                ))#1D 합성곱층

In [189]:
model3.add(Activation(activation='relu'))

In [190]:
model3.add(Dropout(0.5))

In [191]:
model3.add(keras.layers.GlobalMaxPooling1D())

In [192]:
model3.add(Flatten()) #완전 연결 계층
model3.add(Dense(128, activation = 'relu'))
model3.add(Dropout(0.1))
model3.add(Dense(classes, activation = 'softmax'))

In [193]:
model3.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 256, 300)          10051800  
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 248, 64)           172864    
_________________________________________________________________
activation_25 (Activation)   (None, 248, 64)           0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 248, 64)           0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 243, 128)          49280     
_________________________________________________________________
activation_26 (Activation)   (None, 243, 128)          0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 243, 128)        

In [194]:
model3.compile(loss = 'categorical_crossentropy', 
              optimizer = 'Adam',
              metrics = ['acc'])

In [195]:
history = model3.fit(x = X_train, 
                    y = y_train, 
                    batch_size = 64, 
                    epochs = 5, 
                    verbose = 1, 
                    validation_split = 0.1)

Epoch 1/5
772/772 [==============================] - 54s 68ms/step - loss: 1.1973 - acc: 0.4988 - val_loss: 0.9830 - val_acc: 0.6729
Epoch 2/5
772/772 [==============================] - 53s 68ms/step - loss: 0.7269 - acc: 0.7304 - val_loss: 0.8334 - val_acc: 0.7225
Epoch 3/5
772/772 [==============================] - 52s 68ms/step - loss: 0.5244 - acc: 0.8082 - val_loss: 0.7719 - val_acc: 0.7159
Epoch 4/5
772/772 [==============================] - 52s 67ms/step - loss: 0.4157 - acc: 0.8493 - val_loss: 0.7737 - val_acc: 0.7141
Epoch 5/5
772/772 [==============================] - 52s 68ms/step - loss: 0.3389 - acc: 0.8780 - val_loss: 0.8016 - val_acc: 0.7088
